<a href="https://colab.research.google.com/github/JayantJharkhande3000/CS6910_Assignment/blob/main/Part_A_Question_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Question No 2 and 3

Importing all packages and libraries

In [1]:
pip install wandb

     |████████████████████████████████| 1.8 MB 4.1 MB/s 
     |████████████████████████████████| 181 kB 47.0 MB/s 
     |████████████████████████████████| 144 kB 49.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=096bb740a5643b948f90c3ec2a8ee8c9207c6db83736cabc49ab4c3c4b116d81
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [2]:
!pip install wandb --upgrade

In [3]:
import wandb
from wandb.keras import WandbCallback

In [4]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
wandb.init(project="Final_PC_on_fire", entity="jayant3000")

wandb: Currently logged in as: jayant3000 (use `wandb login --relogin` to force relogin)


In [6]:
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from PIL import Image

Importing inaturalist_12K data

In [7]:
data_dirrectory_path = pathlib.Path('/content/drive/MyDrive/DATA/nature_12K/inaturalist_12K') # Set path to the right directory
training_data = tf.keras.preprocessing.image_dataset_from_directory(
                                                                 directory = "/content/drive/MyDrive/DATA/nature_12K/inaturalist_12K/train",
                                                                 labels = 'inferred',  
                                                                 label_mode = 'categorical',
                                                                 color_mode = 'rgb',
                                                                 batch_size = 32,
                                                                 image_size = (256, 256),
                                                                 shuffle = True,
                                                                 seed = 17,
                                                                 validation_split = 0.2,
                                                                 subset = 'training')

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
                                                                  directory = "/content/drive/MyDrive/DATA/nature_12K/inaturalist_12K/val",
                                                                  labels = 'inferred',  
                                                                  label_mode = 'categorical',
                                                                  color_mode = 'rgb',
                                                                  batch_size = 32,
                                                                  image_size = (256, 256),
                                                                  shuffle = True,
                                                                  seed = 17,
                                                                  validation_split = 0.2,
                                                                  subset = 'validation')



Found 9999 files belonging to 10 classes.
Using 8000 files for training.
Found 2000 files belonging to 10 classes.
Using 400 files for validation.


Spliting the data in 90-10 ratio

In [8]:
length_train, length_val = len(training_data), len(validation_data)
training_data = training_data.take(int(0.10*length_train))
validation_data = validation_data.take(int(0.10*length_val))

Model Parameter ############

Sweep Dictionary 

In [9]:

sweep_config = {   'name':  'random-test-sweep', 
                   'method': 'random'
               }
sweep_config['metric'] = { 'name': 'val_acc',
                           'goal': 'maximize'
                          }
parameters_dict = {
                   'first_layer_filters': {
                                            'values': [32, 64]
                                           },
                   'filter_org':  {
                                    'values': [ 0.5, 1, 2]
                                   }, 
                   'data_aug':  { 
                                   'values': [False, True]
                                 },
                   'batch_norm': {
                                  'values': [False, True]
                                  }, 
                   'dropout':   {
                                  'values': [0.2, 0.3]
                                 },
                   'kernel_size': {
                                   'values': [3]
                                   },
                   'dense_size': {
                                  'values': [32]
                                  },
                   'activation': {
                                  'values': ['relu']
                                   },
                   'num_epochs': {
                                  'values': [10]
                                   }, 
                   'optimizer': {
                                  'values': ['adam']
                                 },
                   'conv_layers': {
                                    'values': [5]
                                   }
                  }
sweep_config['parameters'] = parameters_dict

Convolutional Neural Network

In [10]:
def convolutional_neural_network( config=sweep_config ):
        ###############################################################################################################
        ######################   calling the hyperparameter in the function 
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'firstLayerFilters_{}_filterOrg_{}_dataAug_{}_batchNorm_{}_dropout_{}_kerSize_{}_denseSize_{}'\
                         .format(config.first_layer_filters, config.filter_org, config.data_aug, config.batch_norm, config.dropout, config.kernel_size, config.dense_size)
        
        ###############################################################################################################
        #####################   Convolutional neural network hyperparameters
        first_layer_filters = config.first_layer_filters 
        filter_org = config.filter_org 
        data_aug = config.data_aug 
        batch_norm = config.batch_norm 
        dropout =  config.dropout 
        kernel_size = config.kernel_size
        dense_size = config.dense_size
        activation = config.activation
        num_epochs = config.num_epochs
        optimizer = config.optimizer
        conv_layers = config.conv_layers         
        
        ###############################################################################################################
        
        inputs = tf.keras.Input(shape = (256, 256, 3))
        x = Rescaling(scale = 1.0/255)(inputs)
        filter_sizes = [int(first_layer_filters*(filter_org**layer_num)) for layer_num in range(conv_layers)]
        

        for layer_num in range(conv_layers):
            x = layers.Conv2D(filters = filter_sizes[layer_num], kernel_size = (kernel_size, kernel_size))(x)
            if batch_norm:
                x = layers.BatchNormalization(axis = -1)(x)
            x = layers.Activation(activation)(x)
            x = layers.MaxPooling2D(pool_size = (2, 2))(x)            
                
        x = layers.Flatten()(x)
        x = layers.Dense(dense_size)(x)
        if batch_norm:
            x = layers.BatchNormalization(axis = -1)(x)
        x = layers.Activation(activation)(x)
        if dropout > 0:
            x = layers.Dropout(rate = dropout)(x) 

        outputs = layers.Dense(10, activation ='softmax')(x)
        model = tf.keras.Model(inputs = inputs, outputs = outputs)
       
        model.compile(  optimizer = optimizer,
                        loss = tf.keras.losses.CategoricalCrossentropy(name = 'loss'),
                        metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])
        
        model_history = model.fit( training_data,
                                   epochs = num_epochs,
                                   validation_data = validation_data, 
                                   callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5),
                                            wandb.keras.WandbCallback()])



Sweep 

In [11]:
sweep_id = wandb.sweep(sweep_config, project="Final_PC_on_fire", entity="jayant3000")
wandb.agent(sweep_id, function = convolutional_neural_network)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: flv85syf
Sweep URL: https://wandb.ai/jayant3000/Final_PC_on_fire/sweeps/flv85syf


wandb: Agent Starting Run: v1o8pgh8 with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: False
wandb: 	dense_size: 32
wandb: 	dropout: 0.3
wandb: 	filter_org: 2
wandb: 	first_layer_filters: 32
wandb: 	kernel_size: 3
wandb: 	num_epochs: 10
wandb: 	optimizer: adam


Epoch 1/10
25/25 [==============================] - 390s 11s/step - loss: 2.4578 - acc: 0.1500 - val_loss: 2.4229 - val_acc: 0.1562 - _timestamp: 1649511823.0000 - _runtime: 396.0000
Epoch 2/10
25/25 [==============================] - 12s 395ms/step - loss: 2.2042 - acc: 0.2000 - val_loss: 2.6773 - val_acc: 0.0625 - _timestamp: 1649511834.0000 - _runtime: 407.0000
Epoch 3/10
25/25 [==============================] - 11s 377ms/step - loss: 2.0476 - acc: 0.2738 - val_loss: 2.4645 - val_acc: 0.1562 - _timestamp: 1649511854.0000 - _runtime: 427.0000
Epoch 4/10
25/25 [==============================] - 11s 379ms/step - loss: 2.0098 - acc: 0.3063 - val_loss: 2.5172 - val_acc: 0.0938 - _timestamp: 1649511875.0000 - _runtime: 448.0000
Epoch 5/10
25/25 [==============================] - 11s 381ms/step - loss: 1.9276 - acc: 0.3200 - val_loss: 2.6575 - val_acc: 0.0312 - _timestamp: 1649511896.0000 - _runtime: 469.0000
Epoch 6/10
25/25 [==============================] - 11s 364ms/step - loss: 1.8490

wandb: Agent Starting Run: e2pz20vg with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 32
wandb: 	dropout: 0.3
wandb: 	filter_org: 1
wandb: 	first_layer_filters: 64
wandb: 	kernel_size: 3
wandb: 	num_epochs: 10
wandb: 	optimizer: adam


...

wandb: Agent Starting Run: n92gwjkw with config:
wandb: 	activation: relu
wandb: 	batch_norm: False
wandb: 	conv_layers: 5
wandb: 	data_aug: False
wandb: 	dense_size: 32
wandb: 	dropout: 0.2
wandb: 	filter_org: 2
wandb: 	first_layer_filters: 32
wandb: 	kernel_size: 3
wandb: 	num_epochs: 10
wandb: 	optimizer: adam


Epoch 1/10
25/25 [==============================] - 15s 492ms/step - loss: 2.3117 - acc: 0.1013 - val_loss: 2.3028 - val_acc: 0.0938 - _timestamp: 1649516726.0000 - _runtime: 24.0000
Epoch 2/10
25/25 [==============================] - 14s 470ms/step - loss: 2.3022 - acc: 0.1075 - val_loss: 2.3047 - val_acc: 0.0938 - _timestamp: 1649516740.0000 - _runtime: 38.0000
Epoch 3/10
25/25 [==============================] - 13s 412ms/step - loss: 2.3013 - acc: 0.1063 - val_loss: 2.3018 - val_acc: 0.0625 - _timestamp: 1649516753.0000 - _runtime: 51.0000
Epoch 4/10
25/25 [==============================] - 12s 401ms/step - loss: 2.2998 - acc: 0.1200 - val_loss: 2.2996 - val_acc: 0.1875 - _timestamp: 1649516765.0000 - _runtime: 63.0000
Epoch 5/10
25/25 [==============================] - 12s 400ms/step - loss: 2.3013 - acc: 0.1163 - val_loss: 2.3024 - val_acc: 0.0625 - _timestamp: 1649516777.0000 - _runtime: 75.0000
Epoch 6/10
25/25 [==============================] - 12s 407ms/step - loss: 2.2931 - a

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rx1tseqb with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	conv_layers: 5
wandb: 	data_aug: True
wandb: 	dense_size: 32
wandb: 	dropout: 0.3
wandb: 	filter_org: 0.5
wandb: 	first_layer_filters: 64
wandb: 	kernel_size: 3
wandb: 	num_epochs: 10
wandb: 	optimizer: adam


Epoch 1/10
22/25 [=========================>....] - ETA: 1s - loss: 2.6602 - acc: 0.0994

wandb: Ctrl + C detected. Stopping sweep.
